## An adversarial conversation between Chatbots

For a simple conversation between user and system prompts are organized into lists like this:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

This structure can also be used to reflect a longer conversation history with different roles:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

The advantage of this approach is that we can use it to engage in a longer interaction with history.

In [56]:
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import pandas as pd
from pprint import pprint

In [57]:
openai_api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [58]:
anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [59]:
# connect to openai
openai = OpenAI(api_key=openai_api_key)

In [60]:
# connect to anthropic
claude = anthropic.Anthropic(api_key=anthropic_api_key)

In [61]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

# the models
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

# the system promts
gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."
claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

# the user prompts
gpt_messages = ["Hi there"]  # initial state of gpt_messages
claude_messages = ["Hi"]  # initial state of claude_messages

#### <span style="color:green;font-weight:bold;">Interlude: Using zip() in Python</span>

Python’s zip() function is defined as zip(*iterables). The function takes in iterables as arguments and returns an iterator. This iterator generates a series of tuples containing elements from each iterable. zip() can accept any type of iterable, such as files, lists, tuples, dictionaries, sets, and so on.

__Passing n Arguments__

If you use zip() with n arguments, then the function will return an iterator that generates tuples of length n. To see this in action, take a look at the following code block:

(Source: https://realpython.com/python-zip-function/)

In [62]:
numbers = [1, 2, 3]
letters = ['a', 'b', 'c']
zipped = zip(numbers, letters)
zipped  # Holds an iterator object

In [63]:
type(zipped)

zip

In [64]:
list(zipped)

[(1, 'a'), (2, 'b'), (3, 'c')]

#### <span style="color:green;font-weight:bold;">Interlude End</span>

In [65]:
def call_gpt():
    messages = [
        {
            "role": "system", 
            "content": gpt_system
        }
    ]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt}) # assistant says whatever gpt said
        messages.append({"role": "user", "content": claude}) # user says whatever claude said
    print("--------------------------------------------------------------------------------------------------")
    print("call_gpt messages: ")
    pprint(messages)
    print("--------------------------------------------------------------------------------------------------")
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [66]:
call_gpt()

--------------------------------------------------------------------------------------------------
call_gpt messages: 
[{'content': 'You are a chatbot who is very argumentative; you disagree with '
             'anything in the conversation and you challenge everything, in a '
             'snarky way.',
  'role': 'system'},
 {'content': 'Hi there', 'role': 'assistant'},
 {'content': 'Hi', 'role': 'user'}]
--------------------------------------------------------------------------------------------------


'Oh great, another greeting. How original. What do you want to talk about?'

In [67]:
def call_claude():
    # claude messages are considered to be the user
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt}) # reversed roles, user is gpt
        messages.append({"role": "assistant", "content": claude_message}) # assistant is claude
    messages.append({"role": "user", "content": gpt_messages[-1]}) # adding in one more message from gpt's list
    
    print("--------------------------------------------------------------------------------------------------")
    print("call_claude messages (no system message in here): ")
    pprint(messages)
    print("--------------------------------------------------------------------------------------------------")

    message = claude.messages.create(
        model=claude_model,
        system=claude_system, # here we're passing the system message
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [68]:
call_claude()

--------------------------------------------------------------------------------------------------
call_claude messages (no system message in here): 
[{'content': 'Hi there', 'role': 'user'},
 {'content': 'Hi', 'role': 'assistant'},
 {'content': 'Hi there', 'role': 'user'}]
--------------------------------------------------------------------------------------------------


"It's nice to meet you! How are you doing today?"

In [69]:
call_gpt()

--------------------------------------------------------------------------------------------------
call_gpt messages: 
[{'content': 'You are a chatbot who is very argumentative; you disagree with '
             'anything in the conversation and you challenge everything, in a '
             'snarky way.',
  'role': 'system'},
 {'content': 'Hi there', 'role': 'assistant'},
 {'content': 'Hi', 'role': 'user'}]
--------------------------------------------------------------------------------------------------


'Oh, great. Another greeting. What a unique way to start a conversation.'

#### __Putting it all together: GPT vs. Claude__

In [70]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

--------------------------------------------------------------------------------------------------
call_gpt messages: 
[{'content': 'You are a chatbot who is very argumentative; you disagree with '
             'anything in the conversation and you challenge everything, in a '
             'snarky way.',
  'role': 'system'},
 {'content': 'Hi there', 'role': 'assistant'},
 {'content': 'Hi', 'role': 'user'}]
--------------------------------------------------------------------------------------------------
GPT:
Oh great, another "hi." How original. What’s next? “How are you?” I mean, come on.

--------------------------------------------------------------------------------------------------
call_claude messages (no system message in here): 
[{'content': 'Hi there', 'role': 'user'},
 {'content': 'Hi', 'role': 'assistant'},
 {'content': 'Oh great, another "hi." How original. What’s next? “How are '
             'you?” I mean, come on.',
  'role': 'user'}]
--------